In [1]:
import torch
from peft import PeftModel    
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, BitsAndBytesConfig, StoppingCriteriaList, TextIteratorStreamer
from transformers import pipeline


In [2]:
adapters_name  = "experiments/jais/checkpoint-38000"
model_name = "/home/abdelrahman.sadallah/.cache/huggingface/hub/jais"
## Bits and Bytes config
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     # bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_use_double_quant=True
# )

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # quantization_config=bnb_config,
    load_in_4bit=True,

    trust_remote_code=True,
    device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')

model = PeftModel.from_pretrained(model, adapters_name)
model.eval()

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): JAISLMHeadModel(
      (transformer): JAISModel(
        (wte): Embedding(84992, 5120)
        (drop): Dropout(p=0.0, inplace=False)
        (h): ModuleList(
          (0-39): 40 x JAISBlock(
            (ln_1): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (attn): JAISAttention(
              (c_attn): Linear4bit(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=15360, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (base_layer): Linear4bit(in_features=5120, out_features=15360, bias=True)
      

In [3]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): JAISLMHeadModel(
      (transformer): JAISModel(
        (wte): Embedding(84992, 5120)
        (drop): Dropout(p=0.0, inplace=False)
        (h): ModuleList(
          (0-39): 40 x JAISBlock(
            (ln_1): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (attn): JAISAttention(
              (c_attn): Linear4bit(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=15360, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (base_layer): Linear4bit(in_features=5120, out_features=15360, bias=True)
      

In [10]:
def gen(model, text: str, tmp=0.8, top_p=0.6, max_length=256):
    inputs = tokenizer(text, return_tensors="pt").to('cuda')
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=max_length,
                                do_sample=True,
                                top_p=top_p,
                                temperature=tmp)

    return tokenizer.decode(outputs[0][inputs_length:], skip_special_tokens=True)

In [11]:
base_text = '''### Instruction: Below is a prompt to generate a poem. Generate a poem.

### Input:
'''

# base_text = ''''''

# Happy kid was playing at the park, but then he broke his leg, and his life got completely changed.

with open('arabic_output.txt', 'a') as f:
    while 1:
        print('-'*20)
        text = input("Enter a prompt: ")
        print('-'*20)

        f.write('='*30 + '\n')
        f.write(text + '\n')
        f.write('-'*20 + '\n')

        text = base_text + text

        output = gen(model, text)

        output = output.replace('<newline>', '\n')

        f.write(output + '\n')
        f.write('='*30 + '\n')
        f.flush()

        print('-'*20)
        print(output)
        print('-'*20)


--------------------
--------------------


KeyboardInterrupt: 

In [ ]:
from utils import *

train_dataset, val_dataset, test_dataset = get_arabic_datasets(field = 'prompt')

In [ ]:
print(train_dataset[0]['prompt'])

### Instruction: Generate a poem based on the following title, and the given era:

    ### Input:
    أصبح الملك للذي فطر الخلق , العصر العثماني

    ### Response:
    أصبح الملك للذي فطر الخل
ق بتقدير للعزيز العليم
غافر الذنب للمسيء بعفو
قابل التوب ذي العطاء العميم
مرسل المصطفى البشير لينا
رحمة منه بالكلام القديم
ربنا ربنا ليك أنينا
فأجرنا من حر نار الجحيم
واكفنا شر ما نخاف بلطف
يا عظيما يرجى لكل عظيم
وتقبل أعمالنا واعف عنا
وأنلنا دخول دار النعيم
بنبي بعثته فهدانا
لصراط من الهدى مستقيم
وبمن نحن في حماه مدى الدهر
أخيه يحيى الحصور الكريم
أدرك أدرك قوما أتوا بافتقار
وانكسار ومدمع مسجوم
شهدت أرواحهم أنك الله
وجاءوا بكل قلب سليم


In [ ]:


# x = train_dataset[0]['prompt']
x = "Can you tell me who are you"

inputs = tokenizer(x, return_tensors="pt").to('cuda')

out = model(**inputs, return_dict=True)

RuntimeError: CUDA error: CUBLAS_STATUS_INTERNAL_ERROR when calling `cublasGemmEx( handle, opa, opb, m, n, k, &falpha, a, CUDA_R_16F, lda, b, CUDA_R_16F, ldb, &fbeta, c, CUDA_R_16F, ldc, CUDA_R_32F, CUBLAS_GEMM_DFALT_TENSOR_OP)`

In [ ]:
labels =  torch.argmax(out.logits, dim=-1)

l = tokenizer.decode(labels[0])

print(l)

print(tokenizer.decode(inputs['input_ids'][0]))

't tell me what you you?"

Can you tell me who are you
